## Ideias para talvez usar no futuro...

In [ ]:
# Ideias
# Usar o Rodapé para resolver os materiais faltantes.
# import re
# import sap
# connection = sap.listar_conexoes()
# session = connection.Children(1)
# rodape = session.findById("wnd[0]/sbar").Text # Rodapé
# rodape = rodape.lower()
# padrao = r"material (\d+)"
# correspondencias = re.search(padrao, rodape)
# if correspondencias:
#     # Group 1 retira string 'material'
#     codigo_material = correspondencias.group(1)
#     print(codigo_material)

# Monitor
# from datetime import datetime, timedelta
# import sap
# sessoes = sap.listar_sessoes()
# def monitor204():
#     data = datetime.today().date().strftime('%d/%m/%Y')
#     data = str(data)
#     data = data.replace("/", ".")
#     hora_inicio = datetime.now() - timedelta(hours=1)
#     hora_inicio = hora_inicio.time()
#     hora_inicio = hora_inicio.strftime('%H:%M:%S')
#     hora_fim = datetime.now().strftime('%H:%M:%S')
#
#     session_monitor = sap.criar_sessao(sessoes)
#     session_monitor.StartTransaction("ZSBMM204")
#     session_monitor.findById("wnd[0]/usr/ctxtS_DATA-LOW").Text = data
#     session_monitor.findById("wnd[0]/usr/ctxtS_DATA-HIGH").Text = data
#     session_monitor.findById("wnd[0]/usr/ctxtS_HORA-LOW").Text = hora_inicio
#     session_monitor.findById("wnd[0]/usr/ctxtS_HORA-HIGH").Text = hora_fim
#     session_monitor.findById("wnd[0]/usr/txtS_ORDEM-LOW").Type
# fazer interação ZSBMM204?
# import sap
# connection = sap.listar_conexoes()
# session = connection.Children(1)
# rodape = session.findById("wnd[0]/sbar").Text # Rodapé
# if "ZSBMM204" in rodape:
#     monitor204()
# import sap
# connection = sap.listar_conexoes()
# session = connection.Children(3)

# import pywintypes
# try:
#    monitor = session.findById("wnd[0]/usr/cntlCO_GRID/shellcont/shell")
#    n_linhas = monitor.RowCount
#    lista_monitor = []
#    for i in range(n_linhas):
#       mensagem = monitor.GetCellValue(i, "MENSAGEM")
#       lista_monitor.append(mensagem)
# except pywintypes.com_error:
#     print("Erro.")
# Material bloqueado no monitor 204
# import re
# lista_monitor = str(lista_monitor)
# lista_monitor = lista_monitor.lower()
# if "bloqueados" in lista_monitor:
#     padrao = r"material (\d+)"
#     correspondencias = re.search(padrao, lista_monitor)
#     if correspondencias:
#         # Group 1 retira string 'material'
#         codigo_material = correspondencias.group(1)
#         print(codigo_material)

In [ ]:
import sap

connection = sap.listar_conexoes()
session = connection.Children(1)
rodape = session.findById("wnd[0]/sbar").Text # Rodapé
salvo = "Ajustes de valoração salvos com sucesso."
if salvo == rodape:
    pass

In [ ]:
# ConfereOS.py
"""Módulo de check-up no SAP"""
from sap_connection import connect_to_sap


def consulta_os(n_os, contrato, unadm):
    """Função para consultar ORDEM na transação ZSBPM020."""
    diametro_ramal = None
    diametro_rede = None
    hidro = None

    session = connect_to_sap()
    session.StartTransaction("ZSBPM020")
    campo_os = session.findById("wnd[0]/usr/ctxtS_AUFNR-LOW")
    campo_os.Text = n_os
    session.findById("wnd[0]/usr/txtS_CONTR-LOW").text = contrato
    session.findById("wnd[0]/usr/txtS_UN_ADM-LOW").text = unadm
    session.findById("wnd[0]").sendVKey(8)

    session = connect_to_sap()
    consulta = session.findById("wnd[0]/usr/cntlGRID1/shellcont/shell")
    status_sistema = consulta.GetCellValue(0, "STTXT")
    status_usuario = consulta.GetCellValue(0, "USTXT")
    # Contagem Grid
    num_etapas_linhas = consulta.RowCount

    for n_etapa, restab in enumerate(range(num_etapas_linhas)):
        restab = consulta.GetCellValue(n_etapa, "ZZLOCAL_RELIGA")  # religação
        if restab is not None:
            relig = restab

    for n_etapa, supressao in enumerate(range(num_etapas_linhas)):
        supressao = consulta.GetCellValue(
            n_etapa, "ZZLOCAL_CORTE")  # Supressão
        if supressao is not None:
            corte = supressao

    for n_etapa in range(num_etapas_linhas):
        tipo_tse = consulta.GetCellValue(
            n_etapa, "ZZTSE",
        )
        if tipo_tse in (
            "253000",
            "254000",
            "255000",
            "262000",
            "263000",
            "265000",
            "266000",
            "268000",
            "269000",
            "280000",
            "284500",
            "286000",
            "502000",
            "505000",
            "508000",
            "561000",
            "565000",
            "569000",
            "581000",
            "539000",
            "539000",
            "585000",
        ):
            # Posição da Rede
            posicao_rede = consulta.GetCellValue(n_etapa, "ZZPOSICAO_REDE")
            break
        posicao_rede = None

    for n_etapa in range(num_etapas_linhas):
        tipo_tse = consulta.GetCellValue(
            n_etapa, "ZZTSE",
        )
        if tipo_tse in (
            "502000",
            "505000",
            "508000",
            "561000",
            "565000",
            "569000",
            "581000",
            "539000",
            "539000",
            "585000",
        ):
            # Profundidade
            profundidade = consulta.GetCellValue(n_etapa, "ZZPROFUNDIDADE")
            break
        profundidade = None

    for n_etapa, operacao in enumerate(range(num_etapas_linhas)):
        operacao = consulta.GetCellValue(n_etapa, "VORNR")

    for n_etapa, hidro_colocado in enumerate(range(num_etapas_linhas)):
        hidro_colocado = consulta.GetCellValue(
            n_etapa, "ZZHIDROMETRO_INSTALADO")
        if hidro_colocado:
            hidro = hidro_colocado
            break

    for n_etapa in range(num_etapas_linhas):
        sap_diametro_ramal = consulta.GetCellValue(
            n_etapa, "ZZDIAMETRO_RAMAL",
        )
        if sap_diametro_ramal in ("DN_100", "DN_150", "DN_200", "DN_250", "DN_300"):
            diametro_ramal = sap_diametro_ramal
            break

    for n_etapa in range(num_etapas_linhas):
        sap_diametro_rede = consulta.GetCellValue(
            n_etapa, "ZZDIAMETRO_REDE",
        )
        if sap_diametro_rede in ("100", "150", "200", "250", "300"):
            diametro_rede = sap_diametro_rede
            break

    session.findById("wnd[0]").sendVKey(3)  # Voltar

    return (
        status_sistema,
        status_usuario,
        corte,
        relig,
        posicao_rede,
        profundidade,
        hidro,
        operacao,
        diametro_ramal,
        diametro_rede,
    )


In [ ]:
def novasp(ordem) -> None:
    """Run thread NOVAS SP."""
    session = connect_to_sap()
    session.StartTransaction("ZSBMM216")
    # Unidade Administrativa
    session.findById("wnd[0]/usr/ctxtP_UND").Text = "344"
    # Contrato NOVASP
    session.findById("wnd[0]/usr/ctxtP_CONT").Text = "4600041302"
    session.findById("wnd[0]/usr/ctxtP_MUNI").Text = "100"  # Município
    sap_ordem = session.findById(
        "wnd[0]/usr/ctxtP_ORDEM")  # Campo ordem
    sap_ordem.Text = ordem
    session.findById("wnd[0]").SendVkey(8)  # Aperta botão F8


In [ ]:
ordem = "2338451142"
contrato = "4600041302"
unadm = "344"
(status_sistema,
status_usuario,
corte,
relig,
posicao_rede,
profundidade,
hidro,
operacao,
diametro_ramal,
diametro_rede) = consulta_os(ordem, contrato, unadm)


In [ ]:
novasp(ordem)

In [ ]:
# ServicosExecutados.py
"""Módulo de TSE"""
import numpy as np
from excel_tbs import load_worksheets
from tsepai import pai_dicionario

(
    lista,
    materiais,
    plan_tse,
    plan_precos,
    planilha,
    contratada,
    unitario,
    rem_base,
    naoexecutado,
    invest,
    tb_contratada,
    tb_contratada_gb,
    tb_tse_un,
    tb_tse_rem_base,
    tb_tse_nexec,
    tb_tse_invest,
    tb_st_sistema,
    tb_st_usuario,
    tb_tse_pertence_ao_servico_principal,
    tb_tse_reposicao,
    tb_tse_retrabalho,
) = load_worksheets()

session = connect_to_sap()
sondagem = [
    "591000",
    "567000",
    "321000",
    "321500",
    "283000",
    "283500",
]
pai_tse = 0
servico = session.findById("wnd[0]/usr/tabsTAB_ITENS_PRECO/tabpTABS/ssubSUB_TAB:"
                            + "ZSBMM_VALORACAOINV:9010/cntlCC_SERVICO/shellcont/shell")
# Lista temporária para armazenar as tse
list_chave_rb_despesa = []
list_chave_unitario = []
tse_temp = []
identificador_list = []
num_tse_linhas = servico.RowCount
rem_base_reposicao = []
unitario_reposicao = []
mae = False
chave_unitario = None
chave_rb_despesa = None
chave_rb_investimento = None
for n_tse, sap_tse in enumerate(range(num_tse_linhas)):
    sap_tse = servico.GetCellValue(n_tse, "TSE")
    etapa_pai = servico.GetCellValue(n_tse, "ETAPA")

    if sap_tse in tb_tse_un:  # Verifica se está no Conjunto Unitários
        servico.modifyCell(n_tse, "PAGAR", "s")  # Marca pagar na TSE
        # Coloca a tse existente na lista temporária
        tse_temp.append(sap_tse)
        # pylint: disable=E1121
        (reposicao,
            tse_proibida,
            identificador,
            etapa_reposicao) = pai_dicionario.pai_servico_unitario(sap_tse)
        identificador_list.append(identificador)
        chave_unitario = sap_tse, etapa_pai, identificador, reposicao, etapa_reposicao
        unitario_reposicao.append(reposicao)
        list_chave_unitario.append(chave_unitario)
        pai_tse += 1
        if tse_proibida is not None:
            break
        continue

    elif sap_tse in sondagem or sap_tse in tb_tse_rem_base:  # Caso Contrário, é RB - Sondagem
        servico.modifyCell(n_tse, "PAGAR", "n")  # Cesta
        servico.modifyCell(n_tse, "CODIGO", "5")  # Despesa
        # Coloca a tse existente na lista temporária
        tse_temp.append(sap_tse)
        (reposicao,
            tse_proibida,
            identificador,
            etapa_reposicao) = pai_dicionario.pai_servico_cesta(sap_tse)
        rem_base_reposicao.append(reposicao)
        identificador_list.append(identificador)
        chave_rb_despesa = sap_tse, etapa_pai, identificador, reposicao, etapa_reposicao
        list_chave_rb_despesa.append(chave_rb_despesa)
        pai_tse += 1
        if tse_proibida is not None:
            break
        continue

    elif sap_tse in tb_tse_invest:  # Caso Contrário, é RB - Investimento
        servico.modifyCell(n_tse, "PAGAR", "n")  # Cesta
        servico.modifyCell(n_tse, "CODIGO", "6")  # Investimento
        # Coloca a tse existente na lista temporária
        tse_temp.append(sap_tse)
        (reposicao,
            tse_proibida,
            identificador,
            etapa_reposicao) = pai_dicionario.pai_servico_cesta(sap_tse)
        rem_base_reposicao.append(reposicao)
        identificador_list.append(identificador)
        chave_rb_investimento = sap_tse, etapa_pai, identificador, reposicao, etapa_reposicao
        pai_tse += 1
        mae = True
        if tse_proibida is not None:
            break
        continue

    # Pulando OS com asfalto incluso.
    elif sap_tse in tb_tse_asfalto:
        tse_proibida = "Aslfato na bagaça!"
        break

    # Reposição de Guia, fazer manual.
    elif sap_tse == "755000":
        tse_proibida = "Reposicao de Guia!"
        break

    # Retirado Entulho.
    elif sap_tse == "714000":
        tse_proibida = "Retirado Entulho"
        break

    # REPOSIÇÃO DE PARALELO , fazer manual.
    elif sap_tse == "782500":
        tse_proibida = "REPOSIÇÃO DE SARJETA INV"
        reposicao = sap_tse
        etapa_reposicao = etapa_pai
        break

    # Compactação e selagem da base.
    elif sap_tse in ("758500", "758000"):
        tse_proibida = "PARALELO"
        break

    # Readequado Cavalete, verificar...
    # elif sap_tse == '138000':
    #     tse_proibida = 'Readequado Cavalete!'
    #     break

    # Suprimido Ramal anterior
    elif sap_tse == "415000":
        tse_proibida = "Ramal anterior"
        break

    # Serviços relacionados a obra.
    elif sap_tse in ("300000", "308000", "310000", "311000", "313000",
                        "315000", "532000", "540000", "564000",
                        "588000", "590000", "709000", "700000", "593000"):
        tse_proibida = "Obra."
        break

    elif sap_tse in tb_tse_nexec:
        servico.modifyCell(n_tse, "PAGAR", "n")  # Cesta
        servico.modifyCell(n_tse, "CODIGO", "11")  # Não Executado
        continue

    elif sap_tse in tb_tse_pertence_ao_servico_principal:
        servico.modifyCell(n_tse, "PAGAR", "n")
        # Pertence ao Serviço Principal
        servico.modifyCell(n_tse, "CODIGO", "3")
        continue

    elif sap_tse in tb_tse_retrabalho:
        servico.modifyCell(n_tse, "PAGAR", "n")
        servico.modifyCell(n_tse, "CODIGO", "7")  # Retrabalho
        continue

    elif sap_tse in ("730600", "730700"):
        # Compactação e Selagem da Base.
        servico.modifyCell(n_tse, "PAGAR", "n")
        servico.modifyCell(n_tse, "CODIGO", "1")  # Divergência
        continue

    elif sap_tse in ("761000", "762000"):
        # REPOSIÇÃO DE PAREDE/MURO INV.
        servico.modifyCell(n_tse, "PAGAR", "n")
        servico.modifyCell(n_tse, "CODIGO", "1")  # Divergência
        continue

    elif sap_tse == "666000":
        # VISTORIADO LOCAL E IDENTIFICADA SITUAÇÃO
        servico.modifyCell(n_tse, "PAGAR", "n")
        servico.modifyCell(n_tse, "CODIGO", "10")  # Serviço MOP
        continue

if tse_proibida is not None:
    reposicao = None
    etapa_reposicao = None
    rem_base_reposicao_union = None
    reposicao_geral = None

rem_base_reposicao_union = np.unique(rem_base_reposicao, axis=None)
unitario_reposicao_flat = np.ravel(unitario_reposicao)



In [ ]:
reposicao_geral = np.unique(np.concatenate(
    [rem_base_reposicao_union, unitario_reposicao_flat]))
if chave_rb_despesa is not None and pai_tse == 1 and chave_rb_despesa[0] in sondagem:
    for n_tse, sap_tse in enumerate(range(num_tse_linhas)):
        sap_tse = servico.GetCellValue(n_tse, "TSE")
        etapa_pai = servico.GetCellValue(n_tse, "ETAPA")
        # Altera todas as reposições de rb para N3 de Sondagem.
        if sap_tse in rem_base_reposicao_union:
            servico.modifyCell(n_tse, "PAGAR", "n")  # Cesta
            # Pertence ao serviço Principal
            servico.modifyCell(n_tse, "CODIGO", "3")

if mae is True:
    for n_tse, sap_tse in enumerate(range(num_tse_linhas)):
        sap_tse = servico.GetCellValue(n_tse, "TSE")
        etapa_pai = servico.GetCellValue(n_tse, "ETAPA")
        # Altera todas as reposições de rb para investimento se tiver tra.
        if sap_tse in rem_base_reposicao_union:
            servico.modifyCell(n_tse, "PAGAR", "n")  # Cesta
            servico.modifyCell(n_tse, "CODIGO", "6")  # Investimento
# Fim da condicional.
servico.pressEnter()


In [ ]:
import sap

sessions = sap.listar_sessoes()
sessao = sap.criar_sessao(sessions)


In [ ]:
# confere_os.py
import asyncio

from sap_connection import connect_to_sap

lock = asyncio.Lock()

async def consulta_os(n_os, contrato, unadm):
    diametro_ramal = None
    diametro_rede = None
    hidro = None

    async def zsbpm020() -> None:
        nonlocal n_os, contrato, unadm
        async with lock:
            session = await connect_to_sap()
            session.StartTransaction("ZSBPM020")
            campo_os = session.findById("wnd[0]/usr/ctxtS_AUFNR-LOW")
            campo_os.Text = n_os
            session.findById("wnd[0]/usr/txtS_CONTR-LOW").text = contrato
            session.findById("wnd[0]/usr/txtS_UN_ADM-LOW").text = unadm
            session.findById("wnd[0]").sendVKey(8)

    async def fetch_data() -> None:
        nonlocal diametro_ramal, diametro_rede, hidro
        session = await connect_to_sap()
        session.findById("wnd[0]/usr/cntlGRID1/shellcont/shell")
        # Resto do código para buscar dados

    # Criar tarefa para zsbpm020
    task_zsbpm020 = asyncio.create_task(zsbpm020())

    # Aguardar a execução de zsbpm020
    await asyncio.gather(task_zsbpm020)

    # Criar tarefa para fetch_data
    task_fetch_data = asyncio.create_task(fetch_data())

    # Aguardar a execução de fetch_data
    await asyncio.gather(task_fetch_data)

    return (
        # Seus resultados aqui
    )


In [ ]:
await consulta_os("", "4600041302", "344")